In [1]:
import itertools
import platform
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns
%precision 3
from matplotlib import pyplot as plt
%matplotlib inline
#그래프를 주피터 놋북에 그리기 위해
import numpy as np
import copy
from matplotlib import ticker
from scipy.stats import probplot
from scipy import stats
import collections
import itertools
import math
#from varname import nameof
import sys
from scipy import stats
from scipy.stats import bernoulli
import scipy
from collections import deque
from sympy import Symbol, solve
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#히스토그램 그리기
# Window
if platform.system() == 'Windows':
    matplotlib.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin': # Mac
    matplotlib.rc('font', family='AppleGothic')
else: #linux
    matplotlib.rc('font', family='NanumGothic')

# 그래프에 마이너스 표시가 되도록 변경
matplotlib.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정
font_location = 'C:/Windows/Fonts/MALGUNSL.TTF' #맑은고딕
font_name = font_manager.FontProperties(fname=font_location).get_name()
rc('font',family=font_name)

In [2]:
import pandas as pd

data_path = '../titanic/'

train = pd.read_csv(data_path + 'train.csv' , index_col = 'PassengerId')
test = pd.read_csv(data_path + 'test.csv' , index_col = 'PassengerId')
submission = pd.read_csv(data_path + 'gender_submission.csv' , index_col= 'PassengerId')

all_data = pd.read_csv(data_path + 'all_data_5_22_17_10_42.csv' , index_col= 'PassengerId')

In [3]:
all_data

,Name,Sex,Embarked,Age,Age_binned,Pclass,SibSp,Parch,Survival,Cabin,Fare,Fare_binned
PassengerId,,,,,,,,,,,,
1,3,1,2,22.000000,20대,3,1,0,사망,8,7.2500,1.9
2,4,0,0,38.000000,30대,1,1,0,생존,2,71.2833,4.2
3,2,0,2,26.000000,20대,3,0,0,생존,8,7.9250,2.0
4,4,0,2,35.000000,30대,1,1,0,생존,2,53.1000,4.0
5,3,1,2,35.000000,30대,3,0,0,사망,8,8.0500,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,2,28.516607,20대,3,0,0,NaN,8,8.0500,2.0
1306,6,0,0,39.000000,30대,1,0,0,NaN,2,108.9000,4.6
1307,3,1,2,38.500000,30대,3,0,0,NaN,8,7.2500,1.9


In [4]:
all_data['log_Fare'] = np.log(all_data['Fare'])
all_data

,Name,Sex,Embarked,Age,Age_binned,Pclass,SibSp,Parch,Survival,Cabin,Fare,Fare_binned,log_Fare
PassengerId,,,,,,,,,,,,,
1,3,1,2,22.000000,20대,3,1,0,사망,8,7.2500,1.9,1.981001
2,4,0,0,38.000000,30대,1,1,0,생존,2,71.2833,4.2,4.266662
3,2,0,2,26.000000,20대,3,0,0,생존,8,7.9250,2.0,2.070022
4,4,0,2,35.000000,30대,1,1,0,생존,2,53.1000,4.0,3.972177
5,3,1,2,35.000000,30대,3,0,0,사망,8,8.0500,2.0,2.085672
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,2,28.516607,20대,3,0,0,NaN,8,8.0500,2.0,2.085672
1306,6,0,0,39.000000,30대,1,0,0,NaN,2,108.9000,4.6,4.690430
1307,3,1,2,38.500000,30대,3,0,0,NaN,8,7.2500,1.9,1.981001


In [5]:
all_data['FareBin'] = pd.qcut(all_data['Fare'], 5)
all_data['Fare_clean'] = all_data['FareBin'].astype('category').cat.codes
all_data

,Name,Sex,Embarked,Age,Age_binned,Pclass,SibSp,Parch,Survival,Cabin,Fare,Fare_binned,log_Fare,FareBin,Fare_clean
PassengerId,,,,,,,,,,,,,,,
1,3,1,2,22.000000,20대,3,1,0,사망,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0
2,4,0,0,38.000000,30대,1,1,0,생존,2,71.2833,4.2,4.266662,"(46.06, 512.329]",4
3,2,0,2,26.000000,20대,3,0,0,생존,8,7.9250,2.0,2.070022,"(7.879, 11.293]",1
4,4,0,2,35.000000,30대,1,1,0,생존,2,53.1000,4.0,3.972177,"(46.06, 512.329]",4
5,3,1,2,35.000000,30대,3,0,0,사망,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,2,28.516607,20대,3,0,0,NaN,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1
1306,6,0,0,39.000000,30대,1,0,0,NaN,2,108.9000,4.6,4.690430,"(46.06, 512.329]",4
1307,3,1,2,38.500000,30대,3,0,0,NaN,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0


In [6]:
all_data['Family'] = all_data['SibSp'] + all_data['Parch']

all_data

,Name,Sex,Embarked,Age,Age_binned,Pclass,SibSp,Parch,Survival,Cabin,Fare,Fare_binned,log_Fare,FareBin,Fare_clean,Family
PassengerId,,,,,,,,,,,,,,,,
1,3,1,2,22.000000,20대,3,1,0,사망,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,1
2,4,0,0,38.000000,30대,1,1,0,생존,2,71.2833,4.2,4.266662,"(46.06, 512.329]",4,1
3,2,0,2,26.000000,20대,3,0,0,생존,8,7.9250,2.0,2.070022,"(7.879, 11.293]",1,0
4,4,0,2,35.000000,30대,1,1,0,생존,2,53.1000,4.0,3.972177,"(46.06, 512.329]",4,1
5,3,1,2,35.000000,30대,3,0,0,사망,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,2,28.516607,20대,3,0,0,NaN,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
1306,6,0,0,39.000000,30대,1,0,0,NaN,2,108.9000,4.6,4.690430,"(46.06, 512.329]",4,0
1307,3,1,2,38.500000,30대,3,0,0,NaN,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,0


In [7]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

all_data['Survival'] = label_encoder.fit_transform(all_data['Survival'])
all_data

,Name,Sex,Embarked,Age,Age_binned,Pclass,SibSp,Parch,Survival,Cabin,Fare,Fare_binned,log_Fare,FareBin,Fare_clean,Family
PassengerId,,,,,,,,,,,,,,,,
1,3,1,2,22.000000,20대,3,1,0,0,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,1
2,4,0,0,38.000000,30대,1,1,0,1,2,71.2833,4.2,4.266662,"(46.06, 512.329]",4,1
3,2,0,2,26.000000,20대,3,0,0,1,8,7.9250,2.0,2.070022,"(7.879, 11.293]",1,0
4,4,0,2,35.000000,30대,1,1,0,1,2,53.1000,4.0,3.972177,"(46.06, 512.329]",4,1
5,3,1,2,35.000000,30대,3,0,0,0,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,2,28.516607,20대,3,0,0,2,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
1306,6,0,0,39.000000,30대,1,0,0,2,2,108.9000,4.6,4.690430,"(46.06, 512.329]",4,0
1307,3,1,2,38.500000,30대,3,0,0,2,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,0


## Cabin 전처리 해보기

In [8]:
all_data['Cabin'].value_counts()

8    1014
2      94
1      65
3      46
4      41
0      22
5      21
6       5
7       1
Name: Cabin, dtype: int64

In [9]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Name         1309 non-null   int64   
 1   Sex          1309 non-null   int64   
 2   Embarked     1309 non-null   int64   
 3   Age          1309 non-null   float64 
 4   Age_binned   1309 non-null   object  
 5   Pclass       1309 non-null   int64   
 6   SibSp        1309 non-null   int64   
 7   Parch        1309 non-null   int64   
 8   Survival     1309 non-null   int32   
 9   Cabin        1309 non-null   int64   
 10  Fare         1309 non-null   float64 
 11  Fare_binned  1309 non-null   float64 
 12  log_Fare     1309 non-null   float64 
 13  FareBin      1309 non-null   category
 14  Fare_clean   1309 non-null   int8    
 15  Family       1309 non-null   int64   
dtypes: category(1), float64(4), int32(1), int64(8), int8(1), object(1)
memory usage: 151.1+ KB


In [10]:
all_data

,Name,Sex,Embarked,Age,Age_binned,Pclass,SibSp,Parch,Survival,Cabin,Fare,Fare_binned,log_Fare,FareBin,Fare_clean,Family
PassengerId,,,,,,,,,,,,,,,,
1,3,1,2,22.000000,20대,3,1,0,0,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,1
2,4,0,0,38.000000,30대,1,1,0,1,2,71.2833,4.2,4.266662,"(46.06, 512.329]",4,1
3,2,0,2,26.000000,20대,3,0,0,1,8,7.9250,2.0,2.070022,"(7.879, 11.293]",1,0
4,4,0,2,35.000000,30대,1,1,0,1,2,53.1000,4.0,3.972177,"(46.06, 512.329]",4,1
5,3,1,2,35.000000,30대,3,0,0,0,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,2,28.516607,20대,3,0,0,2,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
1306,6,0,0,39.000000,30대,1,0,0,2,2,108.9000,4.6,4.690430,"(46.06, 512.329]",4,0
1307,3,1,2,38.500000,30대,3,0,0,2,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,0


In [11]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# 나이 열 변환
all_data['Age_binned'] = label_encoder.fit_transform(all_data['Age_binned'])

all_data


,Name,Sex,Embarked,Age,Age_binned,Pclass,SibSp,Parch,Survival,Cabin,Fare,Fare_binned,log_Fare,FareBin,Fare_clean,Family
PassengerId,,,,,,,,,,,,,,,,
1,3,1,2,22.000000,2,3,1,0,0,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,1
2,4,0,0,38.000000,3,1,1,0,1,2,71.2833,4.2,4.266662,"(46.06, 512.329]",4,1
3,2,0,2,26.000000,2,3,0,0,1,8,7.9250,2.0,2.070022,"(7.879, 11.293]",1,0
4,4,0,2,35.000000,3,1,1,0,1,2,53.1000,4.0,3.972177,"(46.06, 512.329]",4,1
5,3,1,2,35.000000,3,3,0,0,0,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,2,28.516607,2,3,0,0,2,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
1306,6,0,0,39.000000,3,1,0,0,2,2,108.9000,4.6,4.690430,"(46.06, 512.329]",4,0
1307,3,1,2,38.500000,3,3,0,0,2,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,0


## 성능 개선 없이 그냥 베이스라인 모델 돌려보기

In [12]:
object_col = ['Name' , 'Sex' , 'Embarked'  ,'Age_binned', 'Fare_clean' ,  'Cabin'  ]
binary_col = ['Fare' ,'Family', 'Pclass', 'log_Fare'  ]

idx_col = ['Age_binned' , 'Fare_binned']

In [13]:
all_data.columns.tolist()

['Name',
 'Sex',
 'Embarked',
 'Age',
 'Age_binned',
 'Pclass',
 'SibSp',
 'Parch',
 'Survival',
 'Cabin',
 'Fare',
 'Fare_binned',
 'log_Fare',
 'FareBin',
 'Fare_clean',
 'Family']

In [14]:
all_data_2 = all_data[['Name' , 'Sex' , 'Embarked' , 'Age_binned' , 'Fare_clean' , 'Cabin' , 'Family' , 'Pclass']]

all_data_2

,Name,Sex,Embarked,Age_binned,Fare_clean,Cabin,Family,Pclass
PassengerId,,,,,,,,
1,3,1,2,2,0,8,1,3
2,4,0,0,3,4,2,1,1
3,2,0,2,2,1,8,0,3
4,4,0,2,3,4,2,1,1
5,3,1,2,3,1,8,0,3
...,...,...,...,...,...,...,...,...
1305,3,1,2,2,1,8,0,3
1306,6,0,0,3,4,2,0,1
1307,3,1,2,3,0,8,0,3


# 인코딩 및 스케일링 된 피처 합치기

In [15]:
import numpy as np

all_data_2 = all_data_2.astype(np.float32)



all_data_2

,Name,Sex,Embarked,Age_binned,Fare_clean,Cabin,Family,Pclass
PassengerId,,,,,,,,
1,3.0,1.0,2.0,2.0,0.0,8.0,1.0,3.0
2,4.0,0.0,0.0,3.0,4.0,2.0,1.0,1.0
3,2.0,0.0,2.0,2.0,1.0,8.0,0.0,3.0
4,4.0,0.0,2.0,3.0,4.0,2.0,1.0,1.0
5,3.0,1.0,2.0,3.0,1.0,8.0,0.0,3.0
...,...,...,...,...,...,...,...,...
1305,3.0,1.0,2.0,2.0,1.0,8.0,0.0,3.0
1306,6.0,0.0,0.0,3.0,4.0,2.0,0.0,1.0
1307,3.0,1.0,2.0,3.0,0.0,8.0,0.0,3.0


In [16]:
from scipy import sparse
from scipy.sparse import csr_matrix

all_data_sprs = csr_matrix(all_data_2.values)

all_data_sprs

<1309x8 sparse matrix of type '<class 'numpy.float32'>'
	with 8555 stored elements in Compressed Sparse Row format>

In [17]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기

X= all_data_sprs[:num_train] # 0~num_train -1 행
X_test = all_data_sprs[num_train:] # num_train ~ 마지막 행

y = train['Survived'].values

# y_test = test['Survived'].values

In [18]:
print(X.toarray())

[[3. 1. 2. ... 8. 1. 3.]
 [4. 0. 0. ... 2. 1. 1.]
 [2. 0. 2. ... 8. 0. 3.]
 ...
 [2. 0. 2. ... 8. 3. 3.]
 [3. 1. 0. ... 2. 0. 1.]
 [3. 1. 1. ... 8. 0. 3.]]


In [19]:
all_data

,Name,Sex,Embarked,Age,Age_binned,Pclass,SibSp,Parch,Survival,Cabin,Fare,Fare_binned,log_Fare,FareBin,Fare_clean,Family
PassengerId,,,,,,,,,,,,,,,,
1,3,1,2,22.000000,2,3,1,0,0,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,1
2,4,0,0,38.000000,3,1,1,0,1,2,71.2833,4.2,4.266662,"(46.06, 512.329]",4,1
3,2,0,2,26.000000,2,3,0,0,1,8,7.9250,2.0,2.070022,"(7.879, 11.293]",1,0
4,4,0,2,35.000000,3,1,1,0,1,2,53.1000,4.0,3.972177,"(46.06, 512.329]",4,1
5,3,1,2,35.000000,3,3,0,0,0,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,2,28.516607,2,3,0,0,2,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,0
1306,6,0,0,39.000000,3,1,0,0,2,2,108.9000,4.6,4.690430,"(46.06, 512.329]",4,0
1307,3,1,2,38.500000,3,3,0,0,2,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,0


## 블로그 따라잡기

In [20]:
all_data_2

,Name,Sex,Embarked,Age_binned,Fare_clean,Cabin,Family,Pclass
PassengerId,,,,,,,,
1,3.0,1.0,2.0,2.0,0.0,8.0,1.0,3.0
2,4.0,0.0,0.0,3.0,4.0,2.0,1.0,1.0
3,2.0,0.0,2.0,2.0,1.0,8.0,0.0,3.0
4,4.0,0.0,2.0,3.0,4.0,2.0,1.0,1.0
5,3.0,1.0,2.0,3.0,1.0,8.0,0.0,3.0
...,...,...,...,...,...,...,...,...
1305,3.0,1.0,2.0,2.0,1.0,8.0,0.0,3.0
1306,6.0,0.0,0.0,3.0,4.0,2.0,0.0,1.0
1307,3.0,1.0,2.0,3.0,0.0,8.0,0.0,3.0


In [21]:
all_data['Family'] = 1 + all_data['SibSp'] + all_data['Parch']

all_data['Solo'] = (all_data['Family'] ==1)

all_data

,Name,Sex,Embarked,Age,Age_binned,Pclass,SibSp,Parch,Survival,Cabin,Fare,Fare_binned,log_Fare,FareBin,Fare_clean,Family,Solo
PassengerId,,,,,,,,,,,,,,,,,
1,3,1,2,22.000000,2,3,1,0,0,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,2,False
2,4,0,0,38.000000,3,1,1,0,1,2,71.2833,4.2,4.266662,"(46.06, 512.329]",4,2,False
3,2,0,2,26.000000,2,3,0,0,1,8,7.9250,2.0,2.070022,"(7.879, 11.293]",1,1,True
4,4,0,2,35.000000,3,1,1,0,1,2,53.1000,4.0,3.972177,"(46.06, 512.329]",4,2,False
5,3,1,2,35.000000,3,3,0,0,0,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,2,28.516607,2,3,0,0,2,8,8.0500,2.0,2.085672,"(7.879, 11.293]",1,1,True
1306,6,0,0,39.000000,3,1,0,0,2,2,108.9000,4.6,4.690430,"(46.06, 512.329]",4,1,True
1307,3,1,2,38.500000,3,3,0,0,2,8,7.2500,1.9,1.981001,"(3.17, 7.879]",0,1,True


In [22]:
all_data['Name'].value_counts()

3    757
2    260
4    197
1     61
6     18
0      8
5      8
Name: Name, dtype: int64

In [23]:
feature = [
    'Pclass',
    'Sex',
    'SibSp',
    'Parch',
    'Embarked',
    'Family',
    'Name',
    'Solo',
    'Age_binned',
    'Cabin',
    'Fare_clean'
]

label = [
    'Survived',
]

In [24]:
from sklearn.model_selection import train_test_split
#
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기

X_train= all_data[:num_train] # 0~num_train -1 행
X_test = all_data[num_train:] # num_train ~ 마지막 행

y_train = train['Survived'].values

In [60]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

X_train = X_train[feature]
target = train[label]

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

clf = RandomForestClassifier(n_estimators=50, max_depth=6, random_state=0)
cross_val_score(clf, X_train, target, cv=k_fold, scoring='accuracy', ).mean()
# Accuracy
# 0.8271660424469414

C:\Users\andyp\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\andyp\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\andyp\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\andyp\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y wa

0.8226466916354557

In [58]:
x_train = X_train[feature]
x_test = X_test[feature]
y_train = train[label]
#
# clf = RandomForestClassifier(n_estimators=60, max_depth=6, random_state=0)
clf.fit(x_train, y_train)
y_preds = clf.predict(x_test)

y_preds

<ipython-input-58-32fe055954a8>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(x_train, y_train)


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [59]:
import datetime

# 현재 날짜와 시각 구하기
now = datetime.datetime.now()
date_list = [now.month, now.day, now.hour, now.minute, now.second]
# 현재 날짜와 시각 출력하기
print("현재 날짜 및 시각 : ", date_list)

# 제출 파일 생성

submission['Survived'] = y_preds
submission.to_csv(f'submission_Random_blog_{date_list[0]}_{date_list[1]}_{date_list[2]}_{date_list[3]}_{date_list[4]}.csv')

현재 날짜 및 시각 :  [5, 24, 18, 21, 34]


In [28]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기

X_train= all_data[:num_train] # 0~num_train -1 행
X_test = all_data[num_train:] # num_train ~ 마지막 행

y_train = train['Survived'].values


X_train = X_train[feature]
X_test = X_test[feature]
y_train = train[label]

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()

# 그리드 서치를 위한 하이퍼파라미터 그리드 준비
param_grid = {
    'max_depth': [5,10,12],
    'min_samples_split': [1, 2, 3, 5, 7, 10],
    'min_samples_leaf': [4, 5, 6]
}

# 그리드 서치 객체 생성
grid_search = GridSearchCV(dt_model, param_grid, cv=10 , n_jobs=-1)

# 그리드 서치 수행
grid_search.fit(X_train, y_train)

# 최적의 모델 및 파라미터 출력
best_model_grid_search_dc = grid_search.best_estimator_
best_params_grid_search_dc = grid_search.best_params_
print("Best Model:", best_model_grid_search_dc)
print("Best Parameters:", best_params_grid_search_dc)

Best Model: DecisionTreeClassifier(max_depth=5, min_samples_leaf=6, min_samples_split=3)
Best Parameters: {'max_depth': 5, 'min_samples_leaf': 6, 'min_samples_split': 3}


C:\Users\andyp\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [  nan 0.814 0.809 0.813 0.815 0.811   nan 0.813 0.814 0.815 0.811 0.816
   nan 0.816 0.818 0.814 0.818 0.816   nan 0.813 0.808 0.808 0.81  0.811
   nan 0.808 0.808 0.808 0.81  0.811   nan 0.815 0.811 0.811 0.813 0.813
   nan 0.809 0.813 0.813 0.81  0.818   nan 0.808 0.805 0.808 0.808 0.81
   nan 0.813 0.81  0.813 0.811 0.813]
  warnings.warn(


In [30]:



y_preds = best_model_grid_search_dc.predict(X_test)

y_preds


array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [31]:
print(f'Decsion Tree에 대한 점수 : {best_model_grid_search_dc.score(X_train, y_train)}')

Decsion Tree에 대한 점수 : 0.8395061728395061


In [32]:
# import datetime
#
# # 현재 날짜와 시각 구하기
# now = datetime.datetime.now()
# date_list = [now.month, now.day, now.hour, now.minute, now.second]
# # 현재 날짜와 시각 출력하기
# print("현재 날짜 및 시각 : ", date_list)
#
# # 제출 파일 생성
#
# submission['Survived'] = y_preds
# submission.to_csv(
#     f'submission_gridsearch_DescisionTree_{date_list[0]}_{date_list[1]}_{date_list[2]}_{date_list[3]}_{date_list[4]}.csv')

In [33]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data[:num_train] # 0~num_train-1 행
X_test = all_data[num_train:] # num_train ~ 마지막 행

X_train = X_train[feature]
X_test = X_test[feature]
y_train = train[label]

# 타겟 변수 열을 1차원 배열로 변환
y_train = np.ravel(y_train)
y_train

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# 개별 모델 생성
model_logistic = LogisticRegression()
model_dc_tree = DecisionTreeClassifier()
model_rf = RandomForestClassifier()
model_xgb = xgb.XGBClassifier()
model_lgb = lgb.LGBMClassifier()

# 개별 모델의 하이퍼파라미터 그리드 준비
param_grid_logistic = {'C': [0.1, 0.5, 1.0], 'max_iter': [100, 200, 500] , 'solver':['saga' , 'linear']}
param_grid_dc_tree = {'max_depth': [3, 5, 7], 'min_samples_split': [2, 5, 10]}
param_grid_rf = {'n_estimators': [100, 200, 500], 'max_depth': [None, 5, 10]}
param_grid_xgb = {'max_depth': [3, 5, 7], 'learning_rate': [0.1, 0.01, 0.001]}
param_grid_lgb = {'max_depth': [3, 5, 7], 'learning_rate': [0.1, 0.01, 0.001]}

# 개별 모델의 그리드 서치 객체 생성
grid_search_logistic = GridSearchCV(model_logistic, param_grid_logistic, cv=5)
grid_search_dc_tree = GridSearchCV(model_dc_tree, param_grid_dc_tree, cv=5)
grid_search_rf = GridSearchCV(model_rf, param_grid_rf, cv=5)
grid_search_xgb = GridSearchCV(model_xgb, param_grid_xgb, cv=5)
grid_search_lgb = GridSearchCV(model_lgb, param_grid_lgb, cv=5)



In [62]:

# 개별 모델의 그리드 서치 수행
grid_search_logistic.fit(X_train, y_train)
grid_search_dc_tree.fit(X_train, y_train)
grid_search_rf.fit(X_train, y_train)
grid_search_xgb.fit(X_train, y_train)
grid_search_lgb.fit(X_train, y_train)

C:\Users\andyp\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\andyp\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\andyp\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\andyp\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\andyp\anaconda3\lib\site-packages\sklea

GridSearchCV(cv=5, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [3, 5, 7]})

In [36]:
from sklearn.ensemble import VotingClassifier

# 최적의 모델들 생성
best_model_log = grid_search_logistic.best_estimator_
best_model_dc_Tree = grid_search_dc_tree.best_estimator_
best_model_rf = grid_search_rf.best_estimator_
best_model_xgb = grid_search_xgb.best_estimator_
best_model_lgb = grid_search_lgb.best_estimator_

# 앙상블 모델 생성
ensemble_model = VotingClassifier(
    estimators=[('log', best_model_log),('dcT', best_model_dc_Tree),('rf', best_model_rf), ('xgb', best_model_xgb), ('lgb', best_model_lgb)],
    voting='soft'  # 소프트 보팅 방식 사용 (확률 기반)
)

# 앙상블 모델 훈련
ensemble_model.fit(X_train, y_train)

# 앙상블 모델 예측
ensemble_predictions = ensemble_model.predict(X_test)

C:\Users\andyp\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [37]:
y_preds = ensemble_predictions

y_preds



array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [38]:
print(f'Decsion Tree에 대한 점수 : {ensemble_model.score(X_train, y_train)}')

Decsion Tree에 대한 점수 : 0.8574635241301908


In [39]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegressionCV

num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기

X_train= all_data[:num_train] # 0~num_train -1 행
X_test = all_data[num_train:] # num_train ~ 마지막 행

# y_train = train['Survived'].values

x_train = X_train[feature]
x_test = X_test[feature]
y_train = train[label]
y_train = y_train.values.ravel()

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 개별 모델 생성
model_dc_tree = DecisionTreeClassifier()
model_rf = RandomForestClassifier(n_estimators=50, max_depth=6, random_state=0)
model_xgb = xgb.XGBClassifier()
model_lgb = lgb.LGBMClassifier()
model_logistic = LogisticRegression(C=1.0, max_iter=1000, solver='sag')

# 개별 모델의 하이퍼파라미터 그리드 준비
param_grid_dc_tree = {'max_depth': [3, 5, 7], 'min_samples_split': [2, 5, 10]}
param_grid_xgb = {'max_depth': [3, 5, 7], 'learning_rate': [0.1, 0.01, 0.001]}
param_grid_lgb = {'max_depth': [3, 5, 7], 'learning_rate': [0.1, 0.01, 0.001]}

# 개별 모델의 그리드 서치 객체 생성
grid_search_dc_tree = GridSearchCV(model_dc_tree, param_grid_dc_tree, cv=5)
grid_search_xgb = GridSearchCV(model_xgb, param_grid_xgb, cv=5)
grid_search_lgb = GridSearchCV(model_lgb, param_grid_lgb, cv=5)

# 개별 모델의 그리드 서치 수행
grid_search_dc_tree.fit(x_train, y_train)
grid_search_xgb.fit(x_train, y_train)
grid_search_lgb.fit(x_train, y_train)

# 최적의 개별 모델 및 파라미터 출력
best_model_dc_tree = grid_search_dc_tree.best_estimator_
best_model_xgb = grid_search_xgb.best_estimator_
best_model_lgb = grid_search_lgb.best_estimator_

# 앙상블 모델 생성
ensemble_model = VotingClassifier(
    estimators=[
        ('dcT', best_model_dc_tree),
        ('rf', model_rf),
        ('xgb', best_model_xgb),
        ('lgb', best_model_lgb),
        ('log', model_logistic)
    ],
    voting='hard'
)


In [40]:

# 앙상블 모델 훈련
ensemble_model.fit(x_train, y_train)
print(cross_val_score(ensemble_model, x_train, y_train, cv=k_fold, scoring='accuracy', ).mean())
# 앙상블 모델 예측
ensemble_predictions = ensemble_model.predict(x_test)
print(ensemble_model.score(x_train, y_train))

0.8203870162297129
0.8686868686868687


In [41]:
# import datetime
#
# # 현재 날짜와 시각 구하기
# now = datetime.datetime.now()
# date_list = [now.month, now.day, now.hour, now.minute, now.second]
# # 현재 날짜와 시각 출력하기
# print("현재 날짜 및 시각 : ", date_list)
#
# # 제출 파일 생성
#
# submission['Survived'] = ensemble_predictions
# submission.to_csv(
#     f'submission_gridsearch_ensemble_blog_{date_list[0]}_{date_list[1]}_{date_list[2]}_{date_list[3]}_{date_list[4]}.csv')